In [1]:
import pandas as pd 
import spacy
from email_validator import validate_email, EmailNotValidError
import re
from fuzzywuzzy import process
from metaphone import doublemetaphone
import nltk
from nltk.corpus import words

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
df = pd.read_csv('df.csv')

In [5]:
#Convert columns type to str

df['registrant_name']=df['registrant_name'].astype('str')

### Last Name was mentioned first so correct the sequance and make a new column of Full Name to apply analytics on it

In [6]:
def get_name(name :str) -> str:
    parts = name.split(',')
    if len(parts)==2:
        return f"{parts[1]} {parts[0]}".title()
    else: return name.title()

df['Full_Name'] = df['registrant_name'].apply(get_name)
df
 



,domain_name,create_date,expiry_date,domain_registrar_name,registrant_name,registrant_company,registrant_address,registrant_city,registrant_state,registrant_zip,registrant_country,registrant_email,registrant_phone,Full_Name
0,016naj.us,2/9/2025,2/9/2026,"NameSilo, LLC",Domain Administrator,"NameSilo, LLC",25 N. 23rd Ave Suite 100,Phoenix,AZ,85014,United States,fangyuanhenry20230927@outlook.com,6024928198,Domain Administrator
1,100mg.us,2/9/2025,2/9/2026,Tucows Domains Inc.,Vamani Millhouse,100MG,3678 Scribner Lane,Inglewood,CA,90305,United States,artlandonart@gmail.com,3233779347,Vamani Millhouse
2,120pi.us,2/10/2025,2/10/2030,"NameCheap, Inc.",Jonas Degnan,NaN,PO Box 25731,Honolulu,HI,96825,United States,z0.120pi@gmail.com,6194941118,Jonas Degnan
3,420doc.us,2/9/2025,2/9/2026,PDR Ltd. d/b/a PublicDomainRegistry.com,Diane Alexander,US420DOC,10 Glenlake Pkwy,Atlanta,Georgia,30328,United States,info@us420doc.com,4703562800,Diane Alexander
4,5by9.us,2/9/2025,2/9/2026,"NameCheap, Inc.",David Billsbrough,NaN,560 E 2nd Street,Chuluota,FL,32766,United States,billsbrough@earthlink.net,4073663011,David Billsbrough
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,gadty.com,2/9/2025,2/9/2026,"NameCheap, Inc.",John Doe,NaN,123 Example St,Example City,Example State,12345,United States,bfsdfns@hi2.in,5555555555,John Doe
495,gigerjunkremoval.com,2/9/2025,2/9/2026,"Network Solutions, LLC","Crawford, Lexi",NaN,1318 9TH AVE,HOLDREGE,NE,68949-1907,United States,lgcrawford99@gmail.com,4022192126,Lexi Crawford
496,getservices-inforevision-au.com,2/9/2025,2/9/2026,"OnlineNIC, Inc.",Sally Patrick Patrick,Sally Patrick Patrick,1538 chrisman cir,Oceanside,Califonia,92058,United States,sally5572@yahoo.com,2072345464,Sally Patrick Patrick
497,gotcha-fiq.com,2/9/2025,2/9/2026,"NameCheap, Inc.",Candido Veum,NaN,371 Adams Estates Suite 760 Augusta,Maine,ME,4330,United States,pro@pazdev.net,2545787445,Candido Veum


### **Narrative Report: Name Classification using NLP & Rule-Based Approach**  

#### **Objective:**  
The goal of this process is to accurately classify names into three categories: **Person, Company, and Unknown**. Given a dataset with mixed entity names, the implemented **Natural Language Processing (NLP) and rule-based approach** helps distinguish between company names, individual names (including English, Hindi, and Roman Arabic names), and unknown/gibberish text.

---

### **Methodology:**  

1. **Data Preprocessing & Cleaning:**  
   - Checked for missing values (`NaN`) and marked them as **Unknown**.  
   - Removed special characters and non-alphabetic text to ensure cleaner data.  

2. **Gibberish Detection:**  
   - Defined a function `is_gibberish()` to detect names that are nonsensical or randomly generated.  
   - A name was considered gibberish if:  
     - It had no vowels but excessive consonants.  
     - Contained long non-dictionary words.  
     - Had too many unrecognized words.  

3. **Company Name Identification:**  
   - A predefined **list of company-related keywords** (e.g., "LLC", "Inc.", "Technologies", "Pvt", etc.) was used.  
   - If any of these keywords were found in the name, it was classified as **Company**.  
   - Additionally, entity recognition from `spaCy` was used to detect organizations.  

4. **Person Name Identification:**  
   - Applied **Named Entity Recognition (NER)** using `spaCy` to detect **PERSON** entities.  
   - Used a **set of suffixes** common in **Hindi names** (e.g., "Sharma", "Yadav") and **Arabic names** (e.g., "Bin", "Abdul").  
   - Applied **Double Metaphone Phonetic Matching** to further recognize names based on phonetic similarities.  

5. **Unknown Classification:**  
   - If a name didn’t fit into the **Person** or **Company** category, it was marked as **Unknown**.  
   - Names containing non-alphabetic characters or belonging to a predefined list of **non-name phrases** (e.g., "Admin", "System Generated", "Pending Renewal") were also classified as **Unknown**.  

---

### **Implementation:**  

- The `classify_name()` function was applied to each name in the dataset.  
- The final results were stored in a new column, **"Entity_Type"**, categorizing each entry as **Person, Company, or Unknown**.  

---

### **Results & Accuracy:**  

- **Person Names:** Successfully detected common English, Hindi, and Roman Arabic names.  
- **Company Names:** Identified based on suffixes and `spaCy` entity recognition.  
- **Unknown/Gibberish:** Properly filtered out invalid names and random text.  

This method provides **a robust and automated classification system** for entity identification, improving data integrity for further analysis. 🚀  



In [7]:
nlp = spacy.load('en_core_web_sm')
nltk.download("words")

# Load English word list
english_words = set(words.words())

hindi_suffixes = {"kumar", "sharma", "verma", "yadav", "gupta", "joshi", "bhatt",'das'}
arabic_suffixes = {"al", "bin", "bint", "ibn", "abdul", "faisal", "hassan", "mohammed","ali"}


company_keywords = {"llc", "inc", "ltd", "corp", "technologies", "group", "solutions", "pvt", 
                    "llp", "enterprises",  "advantage", "inc.", "tech",
                    "consulting", "media", "administrator", "consultant", "legal", "amazon","network"}

non_name_phrases = {
    "pending renewal or deletion", "domain for sale", "not available", 
    "system generated", "unknown", "null", "test", "administrator", "marketing", "administration",
                    "customer", "pending", "renewel","delet","support","master", "john doe", "domain", "admin",".com",".net"
}

def is_gibberish(name):
    vowels = set("aeiouAEIOU")
    words = name.split()
    
    if len(name) < 3:  # Too short
        return True

    gibberish_count = 0
    for word in words:
        vowel_count = sum(1 for char in word if char in vowels)
        consonant_count = sum(1 for char in word if char.isalpha() and char not in vowels)

        # If too many consonants without vowels
        if vowel_count == 0 and consonant_count > 5:
            return True

        # If a word is extremely long with no dictionary match, mark as gibberish
        if len(word) > 10 and word.lower() not in english_words: 

            gibberish_count += 1

        # If a word is completely random-looking (regex check for non-dictionary words)
        if re.fullmatch(r"[bcdfghjklmnpqrstvwxyzBCDFGHJKLMNPQRSTVWXYZ]{6,}", word):
            return True
    
    # If more than 50% of words in the name are gibberish, classify as gibberish
    if gibberish_count / len(words) > 0.5:
        return True
    
    return False

def classify_name(name):
    if pd.isna(name) or is_gibberish(name):
        return "Unknown"
    
    if re.search(r'[^a-zA-Z\s]', name):
        return "Unknown"
    
    name_lower = name.lower().strip()

    if any(keyword in name_lower for keyword in non_name_phrases):
        return "Unknown"

    if any(keyword.lower() in name_lower for keyword in company_keywords):
        return "Company"
    
   
    
    doc = nlp(name)

    for ent in doc.ents:
        if ent.label_ == "PERSON":
            return "person"
        elif ent.label_ in {"ORG", "COMPANY"}:
            return "Company"
    
    name_parts = set(name_lower.split())

    if name_parts & hindi_suffixes:
        return 'person'
    elif name_parts & arabic_suffixes:
        return 'person'
    
    metaphone_code = doublemetaphone(name_lower)
    if metaphone_code[0] in {"AKMR", "KMR", "RAHL", "GUPT", "VRM","DAS"}:
        return 'person'
    elif metaphone_code[0] in {"ALFS", "MHM", "ABD", "BN","ALI"}:
        return 'person'
    
        
    return "Unknown"

df['Entity_Type'] = df['Full_Name'].apply(classify_name)


[nltk_data] Downloading package words to /Users/hydermac/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [8]:
def unknown_apply(name):
    if pd.isna(name) or is_gibberish(name):
        return "Unknown"
    
    name_lower = name.lower().strip()

    if any(keyword in name_lower for keyword in company_keywords):
        return 'Company'

    if any(keyword in name_lower for keyword in non_name_phrases):
        return "Unknown"
    return 'person'
    
df.loc[df['Full_Name'].str.lower() != df['registrant_company'].str.lower(),'Entity_Type']='person'
df.loc[df['Full_Name'].isin(['Domain Administrator', 'Domain Admin', 'Account Representative', 'Domain Manager',"Domains Administrators"]), 'Entity_Type'] = 'Unknown'
df.loc[df['Entity_Type'] == 'person', 'Entity_Type'] = df.loc[df['Entity_Type'] == 'person', 'Full_Name'].apply(unknown_apply)


In [22]:
def extract_name(email):
    name_part = email.split("@")[0]
    name_part = re.sub(r'\d+', '', name_part)
    name_part = name_part.replace("."," ").replace("_"," ").replace("-"," ").title().strip()
    return name_part if name_part else None

df.loc[df['Entity_Type'] == 'Unknown', 'Full_Name'] = df.loc[df['Entity_Type'] == 'Unknown', 'registrant_email'].apply(extract_name)


In [24]:
hindi_suffixes = {"kumar", "sharma", "verma", "yadav", "gupta", "joshi", "bhatt",'das'}
arabic_suffixes = {"al", "bin", "bint", "ibn", "abdul", "faisal", "hassan", "mohammed","ali"}


nlp = spacy.load("en_core_web_lg")

def is_human(name):
    if not isinstance(name, str) or not name.strip():
        return "Unknown"
    
    name_lower = name.lower()

    doc = nlp(name)
    for ent in doc.ents:
        if ent.label_ =='PERSON':
            return 'person'

    name_parts = set(name_lower.split())

    if name_parts & hindi_suffixes:
        return 'person'
    elif name_parts & arabic_suffixes:
        return 'person'
    
    metaphone_code = doublemetaphone(name_lower)
    if metaphone_code[0] in {"AKMR", "KMR", "RAHL", "GUPT", "VRM","DAS"}:
        return 'person'
    elif metaphone_code[0] in {"ALFS", "MHM", "ABD", "BN","ALI"}:
        return 'person'
    
    return 'Unknown'

df.loc[df['Entity_Type'] == 'Unknown', 'Entity_Type'] = df.loc[df['Entity_Type']=='Unknown','Full_Name'].apply(is_human)

In [40]:

df_2 = df[0:10]
def is_valid_email(email):
    try:
        validate_email(email, check_deliverability=True)
        return True
    except EmailNotValidError as e:
        print(f"Invalid email: {e}")
        return False
    
df_2['Email_validity'] = df_2['registrant_email'].apply(is_valid_email)
df_2
    


/var/folders/6k/9lcwl8q960scjdl32lqz8h_80000gn/T/ipykernel_92845/3938396378.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['Email_validity'] = df_2['registrant_email'].apply(is_valid_email)


,domain_name,create_date,expiry_date,domain_registrar_name,registrant_name,registrant_company,registrant_address,registrant_city,registrant_state,registrant_zip,registrant_country,registrant_email,registrant_phone,Full_Name,Entity_Type,Email_validity
0,016naj.us,2/9/2025,2/9/2026,"NameSilo, LLC",Domain Administrator,"NameSilo, LLC",25 N. 23rd Ave Suite 100,Phoenix,AZ,85014,United States,fangyuanhenry20230927@outlook.com,6.024928e+09,Domain Administrator,Unknown,True
1,100mg.us,2/9/2025,2/9/2026,Tucows Domains Inc.,Vamani Millhouse,100MG,3678 Scribner Lane,Inglewood,CA,90305,United States,artlandonart@gmail.com,3.233779e+09,Vamani Millhouse,person,True
2,120pi.us,2/10/2025,2/10/2030,"NameCheap, Inc.",Jonas Degnan,NaN,PO Box 25731,Honolulu,HI,96825,United States,z0.120pi@gmail.com,6.194941e+09,Jonas Degnan,person,True
3,420doc.us,2/9/2025,2/9/2026,PDR Ltd. d/b/a PublicDomainRegistry.com,Diane Alexander,US420DOC,10 Glenlake Pkwy,Atlanta,Georgia,30328,United States,info@us420doc.com,4.703563e+09,Diane Alexander,person,True
4,5by9.us,2/9/2025,2/9/2026,"NameCheap, Inc.",David Billsbrough,NaN,560 E 2nd Street,Chuluota,FL,32766,United States,billsbrough@earthlink.net,4.073663e+09,David Billsbrough,person,True
5,a2assementincome.us,2/9/2025,2/9/2026,"NameSilo, LLC",swoosh finance,NaN,8 the green ste r,dover county,DE,19901,United States,swoosh.fin@gmail.com,3.522340e+09,Swoosh Finance,person,True
6,aaha.us,2/10/2025,2/10/2026,Dominet (HK) Limited,Callum Frost,Callum Frost,729 Bayview Ave,Pacific Grove,California,93950,United States,callum@pace.domains,4.156833e+09,Callum Frost,person,True
7,abag.us,2/10/2025,2/10/2026,Dominet (HK) Limited,Kellan Rhodes,Kellan Rhodes,"2847 Elmwood Ave, Apt 12H",Brooklyn,New York,11201,United States,kellan@pace.domains,3.475826e+09,Kellan Rhodes,person,True
8,ablexpress.us,2/10/2025,2/10/2026,"Cloudflare, Inc.",Anthony Lopez,NaN,2072 W 5745 N|Apt C,St. George,UT,84770,United States,anthony1313lopez@gmail.com,9.072023e+09,Anthony Lopez,person,True
9,absolutehealth.us,2/10/2025,2/10/2026,"Spaceship, Inc.",Domain Admin,Brand Consult LLC,1309 Coffeen Avenue STE 1200,Sheridan,WY,82801,United States,brand.consult999@gmail.com,3.072692e+09,Domain Admin,Unknown,True


### **Narrative Report: Email Validity Check**  

#### **Objective:**  
This process verifies whether the **registrant email addresses** are valid and deliverable.  

#### **Steps Taken:**  

1. **Extracting a Sample Dataset**  
   - A subset of the original dataset (`df_2 = df[0:10]`) is created to validate the first **10 email addresses**.  

2. **Email Validation Function (`is_valid_email`)**  
   - The function uses `validate_email()` to check the email’s format and deliverability.  
   - If the email is **valid**, it returns `True`.  
   - If invalid, an error message is displayed, and it returns `False`.  

3. **Applying Validation to Email Column**  
   - The function is applied to `registrant_email` in `df_2`.  
   - A new column, **Email_validity**, is added to store the results (`True` for valid emails, `False` for invalid ones).  

This approach ensures that only valid and deliverable emails are retained for further analysis. ✅

In [9]:
df.to_csv('Email_list.csv',index=False)